# Colorize b/w image

## 1.imports needed

In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, InputLayer
from keras.models import Sequential,load_model
from skimage.color import rgb2lab, lab2rgb, rgb2gray
import numpy as np
import os
import cv2
from skimage.io import imsave
print("done")

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


done


## 2.Gpu threading setting

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

F:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 3.Training list

In [ ]:
X=[]
src = './images/Train/'
i=0
for filename in os.listdir(src):
    if i>1000:
        break;
    i=i+1;
    image = cv2.imread(src+filename)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    X.append(image)
    
Xt = np.array(X,dtype=np.float32)
print("done")

In [ ]:
del X

In [ ]:
X=Xt[:1000]

In [ ]:
del Xt

## 4.Preprocessing

In [ ]:
X=1.0/255*X
Y=rgb2lab(X)[:,:,:,1:] / 128
X=rgb2lab(X)[:,:,:,0]

## 5.Network model

***Build a better architecture***


In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

# Finish model
model.compile(optimizer='adam', loss='mse')

<p>**adam** works better than **rmsprop**</p>
<p>need to find better <strong>loss funtion</strong> probably custom implementation of weighted cross entropy.<p>

In [ ]:
X=X.reshape((1000,256,256,1))
Y=Y.reshape((1000,256,256,2))
model.fit(x=X,y=Y, batch_size=4, epochs=100, verbose=2)

model.save('betagpu-adam')

## Test Code

In [2]:
src='1.jpg'
testimage = cv2.imread(src)
testimage = cv2.resize(testimage,(256,256))
testimage = cv2.cvtColor(testimage, cv2.COLOR_BGR2RGB)
testimage = np.array(testimage,dtype = float)
testimage = rgb2lab(1.0/255*testimage)[:,:,0]
testimage=testimage.reshape((1,256,256,1))
#testimage=X[0]
#testimage=testimage.reshape((1,256,256,1))
model=load_model('betagpu-adam')
output=model.predict(testimage)
output*=128
res=np.zeros((256,256,3))
res[:,:,0]=testimage[0][:,:,0]
res[:,:,1:]=output[0]
imsave("img_result3.png",lab2rgb(res))

error: C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


- Have to print graphs for loss function and accuracy.
- Insertt architecture image.
- any outher documentation part needed

In [29]:
bck=cv2.imread("../Minor2017/res"+src[-6:])
cv2.imwrite("img_result3.png",bck)

True